# 🍷 Wine Quality Prediction using Multiple Linear Regression
## Goal: Predict wine quality (score 0–10) based on physicochemical features.


In [ ]:
# Import libraries
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.metrics import mean_squared_error, r2_score

## Load Data

In [ ]:
df = pd.read_csv('winequality-red.csv')  # Or winequality-white.csv
df.head()